In [1]:
import duckdb
con = duckdb.connect('file.db')

import pandas

# Create a Pandas dataframe
my_df = pandas.DataFrame.from_dict({'a': [42]})

# create the table "my_table" from the DataFrame "my_df"
# Note: duckdb.sql connects to the default in-memory database connection
con.sql("CREATE TABLE my_table AS SELECT * FROM my_df")

# insert into the table "my_table" from the DataFrame "my_df"
con.sql("INSERT INTO my_table SELECT * FROM my_df")

In [2]:
con.sql("SELECT * FROM my_table").show()

┌───────┐
│   a   │
│ int64 │
├───────┤
│    42 │
│    42 │
└───────┘



In [5]:
from sqlalchemy import create_engine
import pandas as pd

connect_args={
        "preload_extensions": ["httpfs"],
        "config": {
          "s3_endpoint":"127.0.0.1:9900",
          "s3_access_key_id":"admin",
          "s3_secret_access_key":"sample_key",
          "s3_url_style":"path",
          "s3_use_ssl":False
  }
}

engine = create_engine("duckdb:///",connect_args=connect_args)

def test_s3():
    sql = "select count(*) from 's3://ontime/*.parquet'"
    df = pd.read_sql(sql, engine)
    print(df)

test_s3()


In [6]:
import duckdb
duckdb.sql('SELECT 42').write_parquet('out.parquet') 

In [14]:
import duckdb

class DuckDBManager:
    def __init__(self, db_name):
        self.db_name = db_name
        self.con = None

    def __enter__(self):
        self.con = duckdb.connect(self.db_name)
        return self.con

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.con.close()


def save_to_db(db_name, table_name, data):
    with DuckDBManager(db_name) as con:
        con.execute(f"DROP TABLE IF EXISTS {table_name}")
        con.sql("CREATE TABLE my_table AS SELECT * FROM data")
        con.sql("SELECT * FROM my_table").show()

In [16]:
import pandas

# Create a Pandas dataframe
my_df = pandas.DataFrame.from_dict({'a': [422]})
save_to_db('file1.db', 'my_table', my_df)

┌───────┐
│   a   │
│ int64 │
├───────┤
│   422 │
└───────┘

